In [ ]:
from autumn.tools.project import get_project
from autumn.settings import Region, Models
from matplotlib import pyplot
from autumn.tools.plots.utils import REF_DATE
import pandas as pd
from autumn.tools.plots.uncertainty.plots import _get_target_values, _plot_targets_to_axis

In [ ]:
# load project
project = get_project(Models.COVID_19, Region.SRI_LANKA)

In [ ]:
# overwrite default parameters with calibrated parameters
calibrated_params = {
    #  list all the calibrated parameters here
    "contact_rate": .1,
    "infectious_seed": 10,
    # ...
}
params = project.param_set.baseline.update(calibrated_params, calibration_format=True)

In [ ]:
# run the model
model = project.run_baseline_model(params)
derived_df = model.get_derived_outputs_df()

In [ ]:
# plot the model outputs and the data used as calibration targets
target_t_min, target_t_max = project.calibration.targets[1].data.index.min(), project.calibration.targets[1].data.index.max()
pyplot.style.use("ggplot")
for output in ["notifications", "infection_deaths"]:    
    fig = pyplot.figure(figsize=(12, 8))
    axis = fig.add_subplot()

    # plot outputs
    axis = derived_df[output].plot()
    axis.set_title(output)

    # Plot calibration targets
    values, times = _get_target_values(project.plots, output)
    values, times = zip(*[(v, t) for (v, t) in zip(values, times) if target_t_min <= t <= target_t_max])    
    date_times = pd.to_datetime(times, origin="01Jan2020",unit="D") 

    _plot_targets_to_axis(axis, values, date_times, on_uncertainty_plot=True)